In [3]:
using Graphs
using DocStringExtensions
using CairoMakie
using RNAStructPlot
using LinearAlgebra

sequence = "CGCUUCAUAUAAUCCUAAUGAUAUGGUUUGGGAGUUUCUACCAAGAGCCUUAAACUCUUGAUUAUGAAGUG"
notation = "...(((((((..((((((.........))))))......).((((((.......))))))..))))))..."

rnabase = dotbracketbase(
    sequence, 
    notation
)

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


RNABaseGraph(SimpleGraph{Int64}(89, [[2], [1, 3], [2, 4], [3, 5, 68], [4, 6, 67], [5, 7, 66], [6, 8, 65], [7, 9, 64], [8, 10, 63], [9, 11, 40]  …  [61, 63], [9, 62, 64], [8, 63, 65], [7, 64, 66], [6, 65, 67], [5, 66, 68], [4, 67, 69], [68, 70], [69, 71], [70]]), Dict{Any, Char}(5 => 'U', 56 => 'U', 16 => 'U', 20 => 'G', 35 => 'U', 55 => 'C', 60 => 'G', 30 => 'G', 19 => 'U', 32 => 'G'…), Dict{Any, Any}(5 => 67, 16 => 30, 7 => 65, 46 => 56, 8 => 64, 17 => 29, 47 => 55, 6 => 66, 43 => 59, 45 => 57…))

In [2]:
function draw_generic(
    rnastruct::RNABaseGraph, drawresult::DrawResult;
    basecolors=Dict('A' => :red,'G' => :cyan,'C' => :yellow,'U' => :lime),
    bondcolor=:red,
    showstrength=true,
    shownumberings=true,
    showstart=true
)

    graph = rnastruct.graph
    bases = rnastruct.nucleotides

    coords = drawresult.coords
    numberings = drawresult.numberings

    f = Figure()
    ax = Axis(f[1,1], aspect = DataAspect())

    # draw edges
    for e in edges(graph)
        sx, sy = coords[src(e)]
        dx, dy = coords[dst(e)]

        # change style if bases are paired
        ispair = haspair(rnastruct, src(e), dst(e))
        edgecolor = ispair ? bondcolor : :black
        strength = showstrength && ispair ? bondstrength(rnastruct, src(e), dst(e)) : 0

        lines!([sx, dx], [sy, dy];linestyle=(:dash, strength^1.8),color=edgecolor)
    end

    # draw vertices with specified coloring
    xs = [coords[v][1] for v in vertices(graph)]
    ys = [coords[v][2] for v in vertices(graph)]
    colors = [basecolors[bases[v]] for v in vertices(graph)]
    scatter!(xs, ys, markersize=20, color=colors)

    # draw nucleotide base labels
    for v in vertices(graph)
        text!(coords[v][1], coords[v][2] ,text=string(bases[v]),align=(:center, :center))
    end
    # draw numbering labels if enabled
    if shownumberings
        for i in vcat([1], range(0,nv(graph), step=5))
            if has_vertex(graph, i) && haskey(coords, i)
                text!(numberings[i][1], numberings[i][2],text=string(i),align=(:center, :center))
            end
        end
    end
    # draw starting arrow pointing to first base in sequence if enabled
    if showstart
        curve = bezier1(coords[1], numberings[1])
        tipx, tipy = arrowtip = curve(0.3)
        tailx, taily = arrowtail = curve(0.8)
        arrows2d!([tailx], [taily], [tipx-tailx], [tipy-taily])
    end
    hidespines!(ax)
    hidedecorations!(ax)
    current_figure()
end

draw_generic (generic function with 1 method)

In [2]:
function layoutcircular(rnabase::RNABaseGraph)
    graph = rnabase.graph
    coords = Dict()
    numberings = Dict()

    # walk along circle
    for (i, vertex) in enumerate(vertices(graph))
        x = (i-1)/nv(graph) * 2 * pi
        coords[vertex] = [cos(x), sin(x)]
        numberings[vertex] = coords[vertex] * 1.1
    end
    return DrawResult(coords, numberings)
end

UndefVarError: UndefVarError: `RNABaseGraph` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [1]:
result = layoutcircular(rnabase)


drawgeneric(rnabase, result)

UndefVarError: UndefVarError: `layoutcircular` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
function draw_bezier(
    rnastruct::RNABaseGraph, drawresult::DrawResult;
    basecolors=Dict('A' => :red,'G' => :cyan,'C' => :yellow,'U' => :lime),
    bondcolor=:red,
    showstrength=true,
    shownumberings=true,
    showstart=true,
    precision=16,
    curvebend=2
)

    graph = rnastruct.graph
    bases = rnastruct.nucleotides

    coords = drawresult.coords
    numberings = drawresult.numberings

    f = Figure()
    ax = Axis(f[1,1], aspect = DataAspect())

    # draw edges
    for e in edges(graph)
        source = sx, sy = coords[src(e)]
        desination = dx, dy = coords[dst(e)]

        # change style if bases are paired
        ispair = haspair(rnastruct, src(e), dst(e))
        edgecolor = ispair ? bondcolor : :black
        strength = showstrength && ispair ? bondstrength(rnastruct, src(e), dst(e)) : 0

        # draw bezier curve
        if ispair
            mid = (source + (desination - source)/2)/curvebend
            bez2 = bezier2(source, mid, desination)

            intervals = 16
            curvex, curvey = [], []
            zerox, zeroy =  bez2(0)
            push!(curvex, zerox)
            push!(curvey, zeroy)

            for i in range(1, intervals)
                x = (i-1)/intervals
                nextx, nexty = bez2(i/intervals)

                push!(curvex, nextx)
                push!(curvey, nexty)
            end
            lines!(curvex, curvey;linestyle=(:dash, strength^1.8), color=edgecolor)
        else
            lines!([sx, dx], [sy, dy];color=edgecolor)
        end
    end

    # draw vertices with specified coloring
    xs = [coords[v][1] for v in vertices(graph)]
    ys = [coords[v][2] for v in vertices(graph)]
    colors = [basecolors[bases[v]] for v in vertices(graph)]
    scatter!(xs, ys, markersize=20, color=colors)

    # draw nucleotide base labels
    for v in vertices(graph)
        text!(coords[v][1], coords[v][2] ,text=string(bases[v]),align=(:center, :center))
    end
    # draw numbering labels if enabled
    if shownumberings
        for i in vcat([1], range(0,nv(graph), step=5))
            if has_vertex(graph, i) && haskey(coords, i)
                text!(numberings[i][1], numberings[i][2],text=string(i),align=(:center, :center))
            end
        end
    end
    # draw starting arrow pointing to first base in sequence if enabled
    if showstart
        curve = bezier1(coords[1], numberings[1])
        tipx, tipy = arrowtip = curve(0.3)
        tailx, taily = arrowtail = curve(0.8)
        arrows2d!([tailx], [taily], [tipx-tailx], [tipy-taily])
    end
    hidespines!(ax)
    hidedecorations!(ax)
    current_figure()
end

draw_bezier(rnabase, result)